In [36]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.contrib.keras as keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# set random seed
np.random.seed(123)
tf.set_random_seed(123)

In [10]:
# import the training data
X_total = pd.read_csv('X_musical_features.csv').drop('track_id', axis=1)

In [80]:
# import single genres y values
y_single_total = pd.read_csv('y_genres_onehot_single.csv').drop('Unnamed: 0', axis=1)

# import non-one-hotted y values
y_cold_single_total = pd.read_csv('y_genres_single.csv')

In [82]:
y_cold_single_total.shape

(63656, 1)

In [23]:
X_total.shape

(63656, 518)

In [22]:
y_single_total.shape

(63656, 139)

In [75]:
X_train, X_test, y_train, y_test = train_test_split(
    X_total, y_single_total, test_size=0.3, shuffle=None
)

# also split y cold 70/30

# then do this for 
# *_train together
# *_test together
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = numpy.random.permutation(len(a))
    return a[p], b[p]

In [76]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [49]:
# del X_train, X_test

In [77]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)

(44559, 518)
(19097, 518)


In [51]:
model = keras.models.Sequential()

In [52]:
model.add(
    keras.layers.Dense(
        units=518,
        input_dim=X_train_scaled.shape[1],
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'
    )
)

Instructions for updating:
Colocations handled automatically by placer.


In [53]:
model.add(
    keras.layers.Dense(
        units=474,
        input_dim=518,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'
    )
)

In [54]:
model.add(
    keras.layers.Dense(
        units=y_train.shape[1],
        input_dim=474,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='softmax'
    )
)

In [55]:
sgd_optimizer = keras.optimizers.SGD(lr=0.001, decay=1e-7, momentum=.9)

In [56]:
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy')

In [105]:
history = model.fit(
    X_train_scaled, y_train,
    batch_size=100, epochs=10,
    verbose=1,
    validation_split=0.1
)

Train on 40103 samples, validate on 4456 samples
Epoch 1/10
40103/40103 [==============================] - 3s 75us/sample - loss: 2.1710 - val_loss: 2.2548
Epoch 2/10
40103/40103 [==============================] - 3s 69us/sample - loss: 2.1553 - val_loss: 2.2521
Epoch 3/10
40103/40103 [==============================] - 3s 70us/sample - loss: 2.1397 - val_loss: 2.2481
Epoch 4/10
40103/40103 [==============================] - 3s 75us/sample - loss: 2.1244 - val_loss: 2.2490
Epoch 5/10
40103/40103 [==============================] - 3s 77us/sample - loss: 2.1094 - val_loss: 2.2455
Epoch 6/10
40103/40103 [==============================] - 3s 78us/sample - loss: 2.0946 - val_loss: 2.2436
Epoch 7/10
40103/40103 [==============================] - 3s 78us/sample - loss: 2.0800 - val_loss: 2.2412
Epoch 8/10
40103/40103 [==============================] - 3s 78us/sample - loss: 2.0658 - val_loss: 2.2399
Epoch 9/10
40103/40103 [==============================] - 3s 80us/sample - loss: 2.0517 - val_l

In [106]:
y_train_pred = model.predict_classes(X_train_scaled, verbose=0)

In [109]:
train_acc = np.sum(y_cold_single_total.tail(44559).values.flatten() == y_train_pred, axis=0) / 44559

In [110]:
print(f"Accuracy on training set: {train_acc}")

Accuracy on training set: 0.08175677192037523
